# Inputs

In [15]:
import sys
sys.path.append('../')
import transport_frames.src.graph_builder.criteria as criteria
import transport_frames.src.metrics.indicators as indicators
import transport_frames.src.metrics.grade_territory as grade_territory
import transport_frames.src.graph_builder.graphbuilder as graphbuilder
import momepy
import osmnx as ox
import geopandas as gpd
import shapely
import pandas as pd
import networkx as nx
import numpy as np
import json

In [16]:
def create_json_file(d1, filename):
    json_dict = {}
    for key in d1.keys():
        if isinstance(d1[key],(float,int)):
            json_dict[key] = d1[key]
        else:
            json_dict[key] = d1[key].to_json()
    json_object = json.dumps(json_dict)
    with open(filename, 'w') as f:
        f.write(json_object)

In [17]:
# Ленинградская_область EPSG: 32636  / 
# Санкт_Петербург EPSG:32636 /
# Москва EPSG:32637 / 
# Волгоградская_область EPSG:32638 / 77665 /  N27504363
# Тульская_область EPSG:32637 /  81993 / N34389350
# Омская_область EPSG:32643 / 140292 /  N27503946
# Краснодарский_край EPSG:32637 / 108082 / N27505129
# Тюменская_область EPSG:32642 / 140291 / N27505666
# Московская_область EPSG:32637 / 

In [18]:
region_name = 'Тульская_область'
local_crs = 32637
geocode = 81993
region_capital = ox.geocode_to_gdf('N34389350',by_osmid=True)

In [19]:
city = ox.geocode_to_gdf(f'R{geocode}', by_osmid=True).to_crs(epsg=4326)
city['name'] = region_name

In [21]:

PLACEHOLDER = gpd.GeoDataFrame(geometry=[]) # set where are no services 

polygons188 = gpd.read_file(f'../data/{region_name}/settlement.geojson')
polygons18 = gpd.read_file(f'../data/{region_name}/district.geojson')

points = gpd.read_file(f'../data/{region_name}/{region_name}_region_points.geojson')
points188 = shapely.centroid(polygons188.geometry).set_crs(polygons188.crs)
points18 = points[points['is_admin_centre_district']==True]


stops = gpd.read_file(f'../data/{region_name}/railway_station.geojson')

fuel = gpd.read_file(f'../data/{region_name}/Копия fuel.geojson')

ferry = PLACEHOLDER

aero = gpd.read_file(f'../data/{region_name}/aerodrome.geojson')

international_aero = aero[aero['aerodrome:type']=='international']
international_aero['geometry'] = shapely.centroid(international_aero['geometry']).set_crs(international_aero.crs)

aero = aero[aero['aerodrome:type']!='international']
aero['geometry'] = shapely.centroid(aero['geometry']).set_crs(aero.crs)


water_objects = gpd.read_file(f'../data/{region_name}/Копия water.geojson')

oopt = gpd.read_file(f'../data/{region_name}/Копия nature_reserve.geojson')

train_paths = gpd.read_file(f'../data/{region_name}/railway.geojson')

bus_stops = gpd.read_file(f'../data/{region_name}/bus_stop.geojson')

russia = ox.geocode_to_gdf("Russia") 
regions = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/Polina/lo_gdfs/regions_of_russia.geojson') #  get regions
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions.to_crs(city.crs)

# test territory
neud = polygons188.iloc[[0]]

# Preprocessing

In [22]:
import os

directory = f"../preprocessed/{region_name}/"
os.makedirs(directory, exist_ok=True)

In [23]:
# write an uds graph
citygraph = graphbuilder.get_graph_from_polygon(city, crs=local_crs,country_polygon=russia)
citygraph_ = graphbuilder.convert_list_attr_to_str(citygraph)
nx.write_graphml(citygraph_, f"../preprocessed/{region_name}/{region_name}_uds.graphml")

/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 13 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


In [24]:
# # write inter into graphml
inter = indicators.get_intermodal(geocode,local_crs)
e = [
        (u, v, k)
        for u, v, k, d in inter.edges(data=True, keys=True)
        if d.get("type") in (['bus','walk'])
    ]
inter = inter.edge_subgraph(e)

# save routes and stops
ni,ei = momepy.nx_to_gdf(inter)
bus_routes = ei[ei['type']=='bus']
bus_routes.to_file(f'../preprocessed/{region_name}/{region_name}_bus_routes.geojson')

# convert to file
inter_= graphbuilder.convert_list_attr_to_str(inter)
for node,e,data in inter_.edges(data=True):
    if 'geometry' in data:
        data['geometry']=str(data['geometry'])
nx.write_graphml(inter_, f"../preprocessed/{region_name}/{region_name}_inter.graphml")

2024-06-14 20:48:34.520 | INFO     | dongraphio.dongraphio:get_intermodal_graph_from_osm:61 - Creating intermodal graph from OSM...
2024-06-14 20:48:34.521 | WARNING  | dongraphio.base_models.grapher_logic:get_public_trasport_graph:237 - Files with routes or with stops was not found. The graph will be built based on data from OSM
2024-06-14 20:48:35.596 | WARNING  | dongraphio.base_models.grapher_logic:public_routes_to_edges:162 - It seems there are no subway routes in the city. This transport type will be skipped.
2024-06-14 20:48:50.117 | INFO     | dongraphio.base_models.grapher_logic:get_public_trasport_graph:285 - Public transport graph done!
2024-06-14 20:48:50.693 | DEBUG    | dongraphio.base_models.grapher_logic:get_osmnx_graph:95 - Extracting and preparing walk graph from OSM ...
/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 12 times your configured Overpass max query area size. It will automa

In [25]:
# write frame into graphml
frame = graphbuilder.get_frame(citygraph,regions.to_crs(city.crs),city)
frame = graphbuilder.assign_city_names_to_nodes(points18.reset_index().to_crs(frame.graph['crs']), 
                                                momepy.nx_to_gdf(frame)[0], frame, name_col='name', max_distance=1200)
frame_ = frame.copy()
for node,e,data in frame_.edges(data=True):
    if 'geometry' in data:
        data['geometry']=str(data['geometry'])
for node,data in frame_.nodes(data=True):
    if 'ref' in data:
        data['ref']=str(data['ref'])
frame_ = graphbuilder.convert_list_attr_to_str(frame_)
nx.write_graphml(frame_, f"../preprocessed/{region_name}/{region_name}_frame.graphml")

/var/folders/ch/1hzm9xv12jg458kt9c3nl6_h0000gn/T/ipykernel_59551/1013225357.py:2: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  frame = graphbuilder.get_frame(citygraph,regions.to_crs(city.crs),city)
/Users/polina/Desktop/github/transport_frames/notebooks/../transport_frames/src/graph_builder/graphbuilder.py:505: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.08]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  n.loc[(n["nodeID"] == path[j]), "weight"] += weight
/Users/polina/Desktop/github/transport_frames/notebooks/../transport_frames/src/graph_builder/graphbuilder.py:506: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.08]' has dtype incompatible with int64, please explicitly cast to a c

# Processing

In [26]:
citygraph = nx.read_graphml(f'../preprocessed/{region_name}/{region_name}_uds.graphml')
citygraph = graphbuilder.convert_list_attr_from_str(citygraph)
citygraph = indicators.prepare_graph(citygraph)

In [27]:
inter = nx.read_graphml(f"../preprocessed/{region_name}/{region_name}_inter.graphml")
inter = graphbuilder.convert_list_attr_from_str(inter)
inter = indicators.prepare_graph(inter)

In [28]:
bus_routes = gpd.read_file(f'../preprocessed/{region_name}/{region_name}_bus_routes.geojson')

# # or if we don't have them:
# ni,ei = momepy.nx_to_gdf(inter)
# bus_routes = ei[ei['type']=='bus']

In [29]:
frame = nx.read_graphml(f"../preprocessed/{region_name}/{region_name}_frame.graphml")
frame = graphbuilder.convert_list_attr_from_str(frame)
frame = indicators.prepare_graph(frame)

In [30]:
d0 = indicators.indicator_area(citygraph, city, points, inter,region_capital, 
                               fuel, stops, international_aero, aero, ferry,train_paths,bus_stops, crs=local_crs)
for k,v in d0.items():
    if isinstance(v,gpd.GeoDataFrame):
        cols_to_drop = [col for col in v.columns if col not in ['index', 'name', 'layer', 'density', 'geometry', 'reg1_length', 'reg2_length', 'reg3_length', 'to_service', 'index', 'number_of_routes', 'service_count', 'total_length_km']]
        v.drop(cols_to_drop, axis=1, inplace=True)
create_json_file(d0,f"../preprocessed/{region_name}/{region_name}_indicators_region.json")

2024-06-14 20:55:14.123 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...
2024-06-14 20:55:15.577 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 20:58:13.669 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 20:58:13.936 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...
2024-06-14 20:58:26.528 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 21:01:40.649 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 21:01:41.078 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...
2024-06-14 21:01:42.475 | DEBUG    | dongraphio.base_models.matrix

Some services cannot be reached from some nodes of the graph. The nodes were removed from analysis


2024-06-14 21:02:38.365 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 21:02:38.864 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 21:02:38.879 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...


Some services cannot be reached from some nodes of the graph. The nodes were removed from analysis


2024-06-14 21:02:40.237 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 21:02:53.825 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 21:02:53.843 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...


...

2024-06-14 21:02:57.235 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 21:03:02.131 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 21:03:02.149 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...


Some services cannot be reached from some nodes of the graph. The nodes were removed from analysis


2024-06-14 21:03:03.504 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 21:03:04.861 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!


Some services cannot be reached from some nodes of the graph. The nodes were removed from analysis


/Users/polina/Desktop/github/transport_frames/notebooks/../transport_frames/src/metrics/indicators.py:402: UserWarning: Approach is not set. Defaulting to 'primal'.
  ei = momepy.nx_to_gdf(inter)[1]


In [31]:
d1 = indicators.indicator_area(citygraph, polygons18, points, inter,region_capital, fuel, stops, international_aero, aero, ferry,train_paths,bus_stops, crs=local_crs)

for k,v in d1.items():
    if isinstance(v,gpd.GeoDataFrame):
        cols_to_drop = [col for col in v.columns if col not in ['index', 'name', 'layer', 'density', 'geometry', 'reg1_length', 'reg2_length', 'reg3_length', 'to_service', 'index', 'number_of_routes', 'service_count', 'total_length_km']]
        v.drop(cols_to_drop, axis=1, inplace=True)
create_json_file(d1,f"../preprocessed/{region_name}/{region_name}_indicators_mr.json")

2024-06-14 21:03:23.230 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...
2024-06-14 21:03:25.899 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 21:06:22.154 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 21:06:22.417 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...
2024-06-14 21:06:32.904 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 21:09:49.074 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 21:09:49.463 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...
2024-06-14 21:09:50.867 | DEBUG    | dongraphio.base_models.matrix

Some services cannot be reached from some nodes of the graph. The nodes were removed from analysis


2024-06-14 21:10:45.288 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 21:10:45.766 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 21:10:45.780 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...


Some services cannot be reached from some nodes of the graph. The nodes were removed from analysis


2024-06-14 21:10:47.146 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 21:11:00.838 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 21:11:00.880 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...


...

2024-06-14 21:11:04.655 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 21:11:09.876 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 21:11:09.895 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...


Some services cannot be reached from some nodes of the graph. The nodes were removed from analysis


2024-06-14 21:11:12.836 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 21:11:14.373 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!


Some services cannot be reached from some nodes of the graph. The nodes were removed from analysis


/Users/polina/Desktop/github/transport_frames/notebooks/../transport_frames/src/metrics/indicators.py:402: UserWarning: Approach is not set. Defaulting to 'primal'.
  ei = momepy.nx_to_gdf(inter)[1]


In [32]:
d2 = indicators.indicator_area(citygraph, polygons188, points, inter,region_capital, 
                               fuel, stops, international_aero, aero, ferry,train_paths,bus_stops, crs=local_crs)

for k,v in d2.items():
    if isinstance(v,gpd.GeoDataFrame):
        cols_to_drop = [col for col in v.columns if col not in ['index', 'name', 'layer', 'density', 'geometry', 'reg1_length', 'reg2_length', 'reg3_length', 'to_service', 'index', 'number_of_routes', 'service_count', 'total_length_km']]
        v.drop(cols_to_drop, axis=1, inplace=True)
create_json_file(d2,f"../preprocessed/{region_name}/{region_name}_indicators_gpsp.json")

/Users/polina/Desktop/github/transport_frames/notebooks/../transport_frames/src/metrics/indicators.py:293: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result = polygons_gdf.merge(length_sums, how='left', left_on='index', right_on='index').fillna(0)
2024-06-14 21:11:32.736 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...
2024-06-14 21:11:34.172 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 21:14:32.231 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 21:14:32.504 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided gr

In [ ]:
d2.keys()

dict_keys(['aggregated_density', 'road_length_gdf', 'connectivity', 'connectivity_public_transport', 'to_fed_roads', 'to_region_admin_center', 'azs_availability', 'number_of_fuel_stations', 'train_stops_availability', 'number_of_train_stops', 'international_aero_availability', 'number_of_international_aero', 'local_aero_availability', 'number_of_local_aero', 'number_of_bus_stops', 'train_paths_length', 'number_of_bus_routes'])

In [ ]:

city

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((41.16810 50.77451, 41.17850 50.77298...",51.247608,47.443733,47.431637,41.168105,172192416,relation,77665,49.604834,44.290358,boundary,administrative,8,0.581413,state,Волгоградская_область,"Volgograd Oblast, Southern Federal District, R..."


In [ ]:
d3 = indicators.indicator_territory(citygraph, neud,
                                    regions_gdf=polygons18,
                                    districts_gdf=polygons188,
                                    region_capital=region_capital,
                                    region_centers=points18,
                                    district_centers=polygons188,
                                    settlement_centers=points,
                                    fuel=fuel,
                                    train_stops=stops,
                                    international_aero=international_aero,
                                    aero=aero,
                                    ports=ferry,
                                    water_objects=water_objects, 
                                    oopt=oopt, 
                                    train_paths=train_paths, 
                                    inter = None,
                                    bus_stops=bus_stops, 
                                    bus_routes=bus_routes,crs=local_crs)

2024-06-14 20:34:33.616 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...
2024-06-14 20:34:41.735 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 20:34:41.936 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 20:34:41.941 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...
2024-06-14 20:34:49.356 | DEBUG    | dongraphio.base_models.matrix_logic:get_adjacency_matrix:52 - Calculating distances from buildings to services ...
2024-06-14 20:34:49.550 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:99 - Adjacency matrix done!
2024-06-14 20:34:49.566 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:90 - Creating adjacency matrix based on provided graph...
2024-06-14 20:34:59.954 | DEBUG    | dongraphio.base_models.matrix

# end